# 1. Model Training

### ⚖️ Check Class Balance
**Displays the number and percentage of images per class.**

*(Ensuring that each class has a similar amount of images will improve the model)*

In [2]:
import os

def check_balance(data_dir='./data'):
    counts = {}
    folders = []  
    
    for class_dir in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            folders.append(class_dir)
            counts[class_dir] = len(os.listdir(class_path))
    
    print("Counts:", counts)
    print("Ratios:", {k: f"{(counts[k]/sum(counts.values())*100):.1f}%" for k in folders})

check_balance()

Counts: {'Finished': 438, 'Opened': 287, 'Sealed': 314}
Ratios: {'Finished': '42.2%', 'Opened': '27.6%', 'Sealed': '30.2%'}


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

CONFIG = {
    'IMG_SIZE': (75, 75),
    'BATCH_SIZE': 32, 
    'EPOCHS': 20,
    'NUM_CLASSES': 3,
    'DATA_DIR': './data'
}

for config in CONFIG:
    print(config)
print("hi")

IMG_SIZE
BATCH_SIZE
EPOCHS
NUM_CLASSES
DATA_DIR
hi


In [4]:

class_names = ['Finished', 'Opened', 'Sealed']

def check_dataset_balance(data_dir):
    """Verify class distribution before training."""
    counts = {}
    for class_dir in sorted(os.listdir(data_dir)):  # Consistent order
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            counts[class_dir] = len([f for f in os.listdir(class_path) 
                                   if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    
    total = sum(counts.values())
    print("Dataset Balance:")
    print(counts)
    print("Ratios:", {k: f"{v/total*100:.1f}%" for k,v in counts.items()})
    return counts

def get_image_generators(data_dir, img_size, batch_size):
    """Simplified 80/10/10 split - more reliable than validation_split trick."""
    seed = 42
    
    # Train: augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # 80/20 split
    )
    
    train_gen = train_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='training', seed=seed
    )
    
    # Val/Test: no augmentation
    val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.125)  # 10% val, 10% test
    
    val_gen = val_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='training', seed=seed, shuffle=False
    )
    
    test_gen = val_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='validation', seed=seed, shuffle=False
    )
    
    return train_gen, val_gen, test_gen

def build_improved_model(img_shape, num_classes):
    """Better architecture: GlobalAvgPool2D + more layers."""
    base_model = InceptionV3(input_shape=(*img_shape, 3), include_top=False, weights='imagenet')
    
    # Freeze base
    base_model.trainable = False
    
    # Better head (no Flatten = less params)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# MAIN TRAINING PIPELINE
print("🔍 Checking dataset...")
check_dataset_balance(CONFIG['DATA_DIR'])

print("\n📊 Loading data...")
train_gen, val_gen, test_gen = get_image_generators(
    CONFIG['DATA_DIR'], CONFIG['IMG_SIZE'], CONFIG['BATCH_SIZE']
)

print("\n🏗️ Building model...")
model = build_improved_model(CONFIG['IMG_SIZE'], CONFIG['NUM_CLASSES'])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# model.summary()

# Compute class weights for imbalance (52%/35%/13% → weights adjust)
print("\n⚖️ Computing class weights...")
class_weights = compute_class_weight(
    'balanced', classes=np.unique(train_gen.classes), y=train_gen.classes
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy'),
    ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-7)
]

print("\n🚀 Training...")
history = model.fit(
    train_gen,
    epochs=CONFIG['EPOCHS'],
    validation_data=val_gen,
    class_weight=class_weight_dict,  # Handles your imbalance!
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Test accuracy:", model.evaluate(test_gen)[1])

# Save with timestamp
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model.save(f'./models/cup_noodle_classifier_{timestamp}.keras')
print(f"💾 Saved: cup_noodle_classifier_{timestamp}.keras")

🔍 Checking dataset...
Dataset Balance:
{'Finished': 438, 'Opened': 287, 'Sealed': 314}
Ratios: {'Finished': '42.2%', 'Opened': '27.6%', 'Sealed': '30.2%'}

📊 Loading data...
Found 833 images belonging to 3 classes.
Found 911 images belonging to 3 classes.
Found 128 images belonging to 3 classes.

🏗️ Building model...

⚖️ Computing class weights...
Class weights: {0: np.float64(0.7910731244064577), 1: np.float64(1.2072463768115942), 2: np.float64(1.1018518518518519)}

🚀 Training...
Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 122s 4s/step - accuracy: 0.5990 - loss: 1.2552 - val_accuracy: 0.8101 - val_loss: 0.4866 - learning_rate: 0.0010
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 111s 4s/step - accuracy: 0.7239 - loss: 0.8375 - val_accuracy: 0.7475 - val_loss: 0.5711 - learning_rate: 0.0010
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - accuracy: 0.7743 - loss: 0.6203 - val_accuracy: 0.8255 - val_loss: 0.4284 - learning_rate: 0.0010
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy

### ⭐ Save the Model as a .keras File (so it can be reused later)

In [33]:
import os
import sys

if not model: 
    print("There's no model yet!")
    sys.exit()  

# get id for new model
folder = "./models/"
id = len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])
default_filename = f"model{id}"

new_model_filename = input(f"Enter filename (Default: {default_filename}): ")
if new_model_filename.strip() == "":
    new_model_filename = default_filename
tf.keras.models.save_model(model, f'./models/{new_model_filename}.keras')

# 2. Trying the Model

### 🔃 Load a .keras model
Loading a model is necessary if the above code to train a new model was not executed.
(Skip this if model was recently trained above)

In [5]:
import os
import tensorflow as tf

folder = "./models/"
files = os.listdir(folder)

# sort by last added time
if not files: raise FileNotFoundError("No models found!")
files_sorted = sorted(files, key=lambda f: os.path.getmtime(os.path.join(folder, f)))
last_added_filename = files_sorted[-1]

while True:
    input_filename = input(
        f"Enter model to load (Latest: {last_added_filename.replace('.keras', '')}): "
    ).strip()

    if input_filename == "":
        model_filename = last_added_filename
        break

    model_filename = f"{input_filename.replace('.keras', '')}.keras"
    full_path = os.path.join(folder, model_filename)

    if os.path.exists(full_path):
        break
    else:
        print("❌ Model not found. Please try again.")

model = tf.keras.models.load_model(f'./models/{model_filename}')

print("✅ Model Loaded:", model_filename)

✅ Model Loaded: cup_noodle_classifier_20260129_113032.keras


### Part 2A: Inference using Camera Stream 📷 
(Please view the cell under this one to terminate the camera)

In [7]:
import threading
import cv2
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf

def import_and_predict(image_data):
    image = ImageOps.fit(image_data, (75, 75), Image.Resampling.LANCZOS)
    image = image.convert('RGB')
    image = np.asarray(image)
    image = (image.astype(np.float32) / 255.0)
    img_reshape = image[np.newaxis,...]
    prediction = model.predict(img_reshape, verbose=0)
    return prediction

stop_flag = False

def camera_loop():
    global stop_flag
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        cap.open()
        
    print("Camera Started!")

    while not stop_flag:
        ret, original = cap.read()
        if not ret:
            break

        image = Image.fromarray(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        prediction = import_and_predict(image)

        map = {0: "Finished",
               1: "Opened",
               2: "Sealed"}

        idx = np.argmax(prediction)
        confidence = prediction[0][idx] 

        predict = f"It is a {map[idx]}! ({confidence:.2f})"

        cv2.putText(original, predict, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.imshow("Cup Noodles Detector", original)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_flag = True

    cap.release()
    cv2.destroyAllWindows()

# start the camera in a separate thread
camera_thread = threading.Thread(target=camera_loop)
camera_thread.start()


Camera Started!


✋ Run to terminate the camera!
   👇

In [8]:
stop_flag = True
camera_thread.join()
print("Stopped!")

Stopped!


### Part 2B: Inference using Folder of Images
For this part, we will classify your images based on uploaded images

In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

COLUMNS = 5

def select_images_with_tkinter():
    """
    Opens tkinter file dialog to select multiple image files.
    Returns list of full file paths.
    """
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    
    # Open file dialog for multiple images
    file_paths = filedialog.askopenfilenames(
        title="Select images to predict",
        filetypes=[
            ("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff *.gif"),
            ("All files", "*.*")
        ]
    )
    
    root.destroy()
    return list(file_paths)

def visualize_predictions_from_files(image_paths: list, class_names: list[str]) -> None:
    """
    Visualize predictions from a list of image file paths (from tkinter).
    """
    if not image_paths:
        print("No images selected.")
        return
    
    rows = math.ceil(len(image_paths) / COLUMNS)
    plt.figure(figsize=(COLUMNS * 4, rows * 4))
    
    for i, img_path in enumerate(image_paths):
        # Load image
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Could not load {img_path}")
            continue
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Preprocess
        img_resized = cv2.resize(img, (75, 75))
        img_norm = img_resized / 255.0
        img_input = np.expand_dims(img_norm, axis=0)
        
        # Predict
        preds = model.predict(img_input, verbose=0)
        class_id = np.argmax(preds[0])
        confidence = preds[0][class_id]
        filename = os.path.basename(img_path)  # Just filename for title
        label = f"{class_names[class_id]} ({confidence:.2f})"
        
        # Plot
        plt.subplot(rows, COLUMNS, i + 1)
        plt.imshow(img)
        plt.title(f"{filename[:20]}...\n{label}", fontsize=10)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Usage - replaces the old line
class_names = ['Finished', 'Opened', 'Sealed']  # Update this for cup noodles!
image_files = select_images_with_tkinter()
visualize_predictions_from_files(image_files, class_names)
